In [1]:
!pip install feedparser -q

     |████████████████████████████████| 81 kB 3.2 MB/s 


In [2]:
import feedparser
from tqdm import tqdm

In [3]:
    def _process(item):
        item = {
            'title': item.get("title", ""),
            'published date': item.get("published", ""),
            'url': item.get("link", ""),
            'publisher': item.get("source", " ")
        }
        return item

    def get_news(key):
        if key != '':
            url = key
            return list(map(_process, feedparser.parse(url).entries[:1000]))

In [4]:
def t_url(keyword,date_after, date_before):
  return "https://news.google.com/rss/search?hl=id&gl=ID&ceid=ID:id&q="+keyword+"+after:"+"-".join(date_after)+"+before:"+"-".join(date_before)

In [5]:
def gen_url_diff(keyword,min_dt, max_dt):
  all_url = []

  # tahun kemarin
  for b in range(12):
    if b<min_dt[1]:
      continue
    for h in range(30):
      if b==min_dt[1] and h<min_dt[2]:
        continue
      #print(x," ",y)
      all_url.append( t_url(keyword, [str(min_dt[0]), str(b), str(h)], [str(min_dt[0]), str(b), str(h+1)] ))
    all_url.append( t_url(keyword, [str(min_dt[0]), str(b), str(31)], ([str(min_dt[0]+1), str(1), str(1)] if b == 12 else [str(min_dt[0]), str(b+1), str(1)]) ))

  # tahun skrg
  for b in range(12):
    if b>max_dt[1]:
      continue
    for h in range(30):
      if b==max_dt[1] and h>max_dt[2]:
        continue
      all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(h)], [str(max_dt[0]), str(b), str(h+1)] ))
    all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(31)], ([str(max_dt[0]+1), str(1), str(1)] if b == 12 else [str(min_dt[0]), str(b+1), str(1)]) ))


  return all_url

In [6]:
def gen_url(keyword,min_dt, max_dt):
  all_url = []

  for b in range(1, 13):
    if b<min_dt[1]:
      continue
    if b>max_dt[1]:
      continue
    for h in range(1, 31):
      if b==min_dt[1] and h<min_dt[2]:
        continue
      if b==max_dt[1] and h>max_dt[2]:
        continue
      all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(h)], [str(max_dt[0]), str(b), str(h+1)] ))
    all_url.append( t_url(keyword, [str(max_dt[0]), str(b), str(31)], ([str(max_dt[0]+1), str(1), str(1)] if b == 12 else [str(min_dt[0]), str(b+1), str(1)]) ))
    if max_dt[2] != 31:
      del all_url[-1]
  return all_url

In [7]:
def run(keyword,min_date,max_date):
  all_url = gen_url_diff(keyword,min_date,max_date) if max_date[0]!=min_date[0] else gen_url(keyword,min_date,max_date)
  
  result = []
  for url in tqdm(all_url):
    result.append(get_news(url))

  return result

In [10]:
keyword   = "tkbm"
min_date  = [2021,9,1]  # t-b-h
max_date  = [2021,9,10] # t-b-h


In [14]:
crawl = run(keyword, min_date, max_date)

crawl_list=[]
count=0
for x in crawl:
  count=count+len(x)
  if x!=[]:
    for key, value in x[0].items():
      temp = [key,value]
      crawl_list.append(temp)
count

100%|██████████| 10/10 [00:04<00:00,  2.36it/s]


18

In [16]:
crawl_list=[]
count=0

for x in crawl:
  count=count+len(x)
  if x!=[]:
    for y in x:
      temp=[]
      for key, value in y.items():
        temp.append(value)
      crawl_list.append(temp)
count

18

In [17]:
import pandas as pd
crawl_df=pd.DataFrame(crawl_list)
crawl_df.columns=['Judul',"Tanggal","Link","Publisher"]
crawl_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Judul      18 non-null     object
 1   Tanggal    18 non-null     object
 2   Link       18 non-null     object
 3   Publisher  18 non-null     object
dtypes: object(4)
memory usage: 704.0+ bytes


In [18]:
crawl_df

,Judul,Tanggal,Link,Publisher
0,"Buka Bimtek, Winarni Pinta Satuan PAUD Bantu M...","Thu, 02 Sep 2021 12:13:45 GMT",https://www.saibumi.com/artikel-112610-buka-bi...,"{'href': 'https://www.saibumi.com', 'title': '..."
1,Pelindo dan Dinkes Provinsi NTT Gelar Vaksinas...,"Sat, 04 Sep 2021 07:00:00 GMT",https://kupang.tribunnews.com/2021/09/04/pelin...,"{'href': 'https://kupang.tribunnews.com', 'tit..."
2,Giliran Buruh Pelabuhan dan Karyawan Besi Tua ...,"Sat, 04 Sep 2021 07:52:21 GMT",https://halmaherapost.com/2021/09/04/giliran-b...,"{'href': 'https://halmaherapost.com', 'title':..."
3,"80 Liter Miras Sopi Ditemukan di Kapal Pelni, ...","Mon, 06 Sep 2021 03:28:00 GMT",https://maluku.inews.id/berita/80-liter-miras-...,"{'href': 'https://maluku.inews.id', 'title': '..."
4,TKBM TBM Minta Gubernur Copot Kadis Koperasi d...,"Tue, 07 Sep 2021 05:52:13 GMT",https://www.kendariaktual.com/tkbm-tbm-minta-g...,"{'href': 'https://www.kendariaktual.com', 'tit..."
5,"Didemo, Kantor Dinas Koperasi dan UMKM Sultra ...","Tue, 07 Sep 2021 05:46:22 GMT",https://www.kendariaktual.com/didemo-kantor-di...,"{'href': 'https://www.kendariaktual.com', 'tit..."
6,Polisi Kembali Sita Miras di Pelabuhan Ambon -...,"Mon, 06 Sep 2021 16:00:07 GMT",https://www.kabartimurnews.com/2021/09/07/poli...,"{'href': 'https://www.kabartimurnews.com', 'ti..."
7,Sat Reskrim Polres Pelabuhan Belawan Ringkus S...,"Mon, 06 Sep 2021 08:24:50 GMT",https://lintaswarta.id/sat-reskrim-polres-pela...,"{'href': 'https://lintaswarta.id', 'title': 'L..."
8,Kemnaker Tingkatkan Pelindungan Bagi Tenaga Ke...,"Tue, 07 Sep 2021 19:38:03 GMT",https://www.liputan6.com/news/read/4652424/kem...,"{'href': 'https://www.liputan6.com', 'title': ..."
9,Kemnaker Tingkatkan Perlindungan Bagi Pekerja ...,"Wed, 08 Sep 2021 01:55:00 GMT",https://m.medcom.id/amp/8KyjwprN-kemnaker-ting...,"{'href': 'https://m.medcom.id', 'title': 'Medc..."


In [ ]:
path="/content/drive/MyDrive/UAS/TKBM/"
#crawl_df.to_csv(path+"crawl.csv",index=False)